In [ ]:
import os # Импортируем модуль os для работы с переменными окружения

# --- Чтение конфигурации из переменной окружения ---

# Используем os.getenv() для получения значения переменной GIGACHAT_API_KEY
# Второй аргумент (None) - это значение по умолчанию, если переменная не установлена
api_key = os.getenv('GIGACHAT_API_KEY', None)

# --- Проверка наличия ключа ---
if not api_key:
    print("Ошибка: Переменная окружения 'GIGACHAT_API_KEY' не установлена.")
    print("Пожалуйста, установите переменную окружения с вашим API ключом GigaChat.")
    # В реальном скрипте здесь, вероятно, нужно остановить выполнение или бросить исключение
    # Для примера оставим print и предположим, что дальше будет какая-то обработка или выход.
    # exit(1) # Пример выхода с ошибкой
else:
    print("[*] API ключ GigaChat успешно прочитан из переменных окружения.")


[*] API ключ GigaChat успешно прочитан из переменных окружения.


In [ ]:
import json

# --- Шаг 1: Загрузите ваши данные (предположим, они в файле 'messages.json') ---
filename = 'parser_results/ejlabru_after_20250417_000000_Europe_Moscow_local.json'
try:
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
except (FileNotFoundError, json.JSONDecodeError) as e:
    print(f"Ошибка при загрузке данных из {filename}: {e}")
    data = [] # Устанавливаем пустой список, чтобы избежать ошибок далее

# --- Шаг 2: Определите параметры ---
# Максимальное количество новостей для включения в промпт
max_news_items = 10 # Можете изменить это значение

# --- Шаг 3: Сформируйте промпт ---

# Список для хранения частей промпта
prompt_parts = []

# 1. Добавляем тексты новостей из загруженных данных
news_counter = 0 # Счетчик для нумерации новостей
for item in data:
    # Проверяем, достигли ли лимита
    if news_counter >= max_news_items:
        print(f"Достигнут лимит в {max_news_items} новостей. Остальные пропускаются.")
        break # Выходим из цикла, если лимит достигнут

    # Получаем текст из словаря, проверяем, что он не пустой
    text = item.get('text')
    if text and text.strip(): # Убедимся, что текст существует и не состоит только из пробелов
        news_counter += 1
        # Формируем блок для текущей новости
        news_block = f"""--- НОВОСТЬ {news_counter} ---
{text}
--- КОНЕЦ НОВОСТИ {news_counter} ---"""
        prompt_parts.append(news_block)
    # else:
        # print(f"Пропущено сообщение с ID {item.get('id')}, т.к. текст отсутствует или пуст.")


# 2. Добавляем заключительную часть промпта
if news_counter > 0: # Добавляем конец, только если были добавлены новости
    # Заключительная часть теперь является основной инструкцией
    final_request = f"""--- КОНЕЦ ВСЕХ НОВОСТЕЙ ---

Пожалуйста, сделай саммари (краткое изложение) представленных выше {news_counter} новост(и/ей). Представь результат в виде маркированного списка ключевых тезисов. Саммари должно быть кратким, но охватывать основные моменты всех статей. Сохраняй нейтральный тон."""
    prompt_parts.append(final_request)
else:
    # Если не было найдено ни одной новости с текстом
    prompt_parts = ["Нет новостей для саммаризации."]


# 3. Объединяем все части в одну строку с переносами строк между блоками
final_llm_prompt = "\n\n".join(prompt_parts)

# --- Шаг 4: Выведите или используйте готовый промпт ---
print(final_llm_prompt)

# Теперь переменную final_llm_prompt можно передавать в ваш LLM API


Достигнут лимит в 1 новостей. Остальные пропускаются.
--- НОВОСТЬ 1 ---
**ЕЖ. Ночь — главные новости к этому часу:**

1️⃣ Стало известно о [задержании](https://t.me/ejdailyru/317341)** **экс-губернатора Курской области Алексея Смирнова и его заместителя Алексея Дедова.** **Они [стали](https://t.me/ejdailyru/317373) фигурантами [уголовного дела](https://t.me/ejdailyru/317394) о мошенничестве на сумму** **как минимум в 1 млрд руб.**
**
2️⃣ ЕС [отложил](https://t.me/ejdailyru/317438) введение запрета на импорт российского СПГ. 

3️⃣ Трамп [продлил](https://t.me/ejdailyru/317389) на год запрет на прием в американских портах судов из России. 

4️⃣ Путин [встретился](https://t.me/ejdailyru/317450) в Кремле с освобожденными из плена в Газе гражданином Израиля и России Александром Труфановым, его матерью и невестой.

5️⃣ В новом правительстве Сербии [не нашлось места](https://t.me/ejdailyru/317435) пророссийскому министру Вулину. **
**
6️⃣ Финляндия [оставила](https://t.me/ejdailyru/317449) гр

In [8]:
"""Пример работы с чатом"""
from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole

payload = Chat(
    messages=[
        Messages(
            role=MessagesRole.SYSTEM,
            content="Пожалуйста, сделай саммари (краткое изложение) следующих новостных статей. Представь результат в виде [выберите один вариант: единого связного абзаца ИЛИ маркированного списка ключевых тезисов]. Саммари должно быть кратким, но охватывать основные моменты всех статей. Сохраняй нейтральный тон."
        )
    ],
    temperature=0.7,
    max_tokens=100,
)

# Используйте токен, полученный в личном кабинете из поля Авторизационные данные
with GigaChat(credentials=api_key, verify_ssl_certs=False, model="GigaChat") as giga:
    # while True:
    user_input = final_llm_prompt
    payload.messages.append(Messages(role=MessagesRole.USER, content=user_input))
    response = giga.chat(payload)
    summary = response.choices[0].message.content


In [9]:
summary


'1. Экс-губернатор Курской области Алексей Смирнов и его заместитель Алексей Дедов задержаны по подозрению в мошенничестве на сумму более 1 млрд рублей.\n2. Евросоюз отложил введение запрета на импорт российского СПГ.\n3. Трамп продлил на год запрет на прием в американских портах российских судов.\n4. Президент России Владимир Путин встретился с освобожденным из плена в Газе граждани'